In [19]:
from sklearn.linear_model import LogisticRegression, ElasticNet, SGDClassifier, MultiTaskElasticNet, BayesianRidge
import pandas as pd
import numpy as np
from sklearn.metrics import RocCurveDisplay, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.decomposition import NMF, PCA


def get_metric(targets, preds):
    auc = roc_auc_score(targets, preds)
    acc = accuracy_score(targets, np.where(preds >= 0.5, 1, 0))
    precsion = precision_score(targets, np.where(preds >= 0.5, 1, 0))
    recall = recall_score(targets, np.where(preds >= 0.5, 1, 0))
    F1_score = f1_score(targets, np.where(preds >= 0.5, 1, 0))

    print('auc :',auc)
    print('acc :',acc)
    print('precision :',precsion)
    print('recall :',recall)

def test_to_csv(preds, name:str):
    
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(f'output/{name}.csv', index=None)
    

In [20]:
cate_cols = [
            # 'assessmentItemID',
            # 'testId',
            # 'KnowledgeTag',
            'hour',
            'dow',
            'i_head',
            'i_mid',
            'i_tail',
]
cont_cols = [                        
            'user_correct_answer',
            'user_total_answer',
            'user_acc',            
            't_elapsed',            
            'cum_correct',
            'last_problem',
            'head_term',
            # 'left_asymptote',
            'elo_prob',
            'pkt',
            'u_head_mean',
            'u_head_count',
            'u_head_std',
            'u_head_elapsed',
            'i_mid_elapsed',
            'i_mid_mean',
            'i_mid_std',
            'i_mid_sum',
            'i_mid_count',
            'i_mid_tag_count',
            'assessment_mean',
            'assessment_sum',
            # 'assessment_std',
            'tag_mean',
            'tag_sum',
            # 'tag_std',
            'tail_mean',
            'tail_sum',
            # 'tail_std',
            'hour_mean',
            'hour_sum',
            # 'hour_std',
            'dow_mean',
            'dow_sum',
            # 'dow_std',
            'tag_elapsed',
            'tag_elapsed_o',
            'tag_elapsed_x',
            'assessment_elapsed',
            'assessment_elapsed_o',
            'assessment_elapsed_x',
            'tail_elapsed',
            'tail_elapsed_o',
            'tail_elapsed_x'
            ]

FEATS = cate_cols + cont_cols

In [21]:
data = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/all.pkl')
label = data.answerCode.to_numpy()

In [31]:
valid = pd.read_csv('/opt/ml/input/data/cv_valid_data.csv').groupby('userID').tail(1)
valid_index = valid[valid.answerCode==-1].index

In [32]:
X_train = data[data.index.isin(valid_index)==False]
y_train = X_train.answerCode.to_numpy()
X_valid = data[data.index.isin(valid_index)==True]
y_valid = X_valid.answerCode.to_numpy()

In [33]:
pca = PCA()

In [34]:
BR = BayesianRidge()

In [35]:
BR.fit(X_train[FEATS].values, y_train)

BayesianRidge()

In [36]:
valid_preds = BR.predict(X_valid[FEATS].values)

In [37]:
get_metric(y_valid,valid_preds)

auc : 0.8674943883277216
acc : 0.8158602150537635
precision : 0.8509874326750448
recall : 0.8977272727272727


In [33]:
test = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/test_data.pkl')
test = test[test.answerCode==-1]

In [1]:
test_preds = BR.predict(test[FEATS].values)

NameError: name 'BR' is not defined

In [36]:
test_to_csv(test_preds,'BR2')

test